In [33]:
#IMPORTO LIBRERIE
import pandas as pd
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

In [34]:
df = pd.read_csv('smote.csv',decimal=',')
df=df.drop(columns='Unnamed: 0')
df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,Italy,150.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
1,Greece,3200.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
2,Italy,8350.00,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0
3,Italy,500.00,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,0
4,Italy,1895.68,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,0


In [35]:
import pickle
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [36]:
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

for clmn in columns_to_label:
    df[clmn]=df[clmn].apply(str)
    df[clmn] = encoder_dict[clmn].transform(df[clmn])

In [37]:
df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,is_anomaly
0,6,150.00,1,4,5,17,699,2013,141,19,113,5,0
1,5,3200.00,1,4,5,17,699,2013,141,19,113,5,0
2,6,8350.00,1,4,5,17,699,2013,141,19,113,5,0
3,6,500.00,1,0,5,17,123,1781,87,8,99,10,0
4,6,1895.68,1,0,5,17,285,1781,87,8,99,10,0


In [42]:
df.isnull().values.any()

False

In [44]:
df.iloc(2)

ValueError: No axis named 2 for object type DataFrame

In [57]:
import numpy as np
from sklearn.model_selection import KFold
X = df.drop(columns='is_anomaly').to_numpy()
y = df['is_anomaly'].to_numpy()
kf = KFold(n_splits=10)
kf.get_n_splits(X)

print(kf)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("TRAIN:", train_index, "TEST:", test_index)
    model=CategoricalNB()
    model.fit(X_train, y_train)
    model.predict(X_test)
    #y_pred = model.predict(X_test)

KFold(n_splits=10, random_state=None, shuffle=False)
TRAIN: [ 18187  18188  18189 ... 181859 181860 181861] TEST: [    0     1     2 ... 18184 18185 18186]
TRAIN: [     0      1      2 ... 181859 181860 181861] TEST: [18187 18188 18189 ... 36371 36372 36373]


IndexError: index 1990000 is out of bounds for axis 1 with size 1254001

In [59]:
from sklearn.model_selection import cross_val_score
#num_folds = 10
kfold = KFold(n_splits=10)
model = CategoricalNB()
results = cross_val_score(model,
                          df.drop('is_anomaly', axis=1),
                          df['is_anomaly'],
                          cv=kfold)
print(results.mean())

C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 108, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 666, in score
    return accuracy_score(y, self.predict(X), 

nan


In [50]:
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [5]:
# store the feature matrix (X) and response vector (y)
#X = df.drop(columns=['is_anomaly','timestamp','businessName','customerId'])
X = df.drop(columns='is_anomaly')
y = df['is_anomaly']

# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# training the model on training set
from sklearn.naive_bayes import BernoulliNB
#gnb = BernoulliNB()
model=CategoricalNB()
model.fit(X_train, y_train)

# making predictions on the testing set
y_pred = model.predict(X_test)

# comparing actual response values (y_test) with predicted response values (y_pred)
from sklearn import metrics
print("Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Naive Bayes model accuracy(in %): 99.37591070299398


In [16]:
test=pd.read_csv('misc_test.csv',decimal=',')
test

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,Austria,264986.74,1,Chrome,Windows,AT,34000,BUV4U,39031,430,161,SRL
1,Italy,111345.77,1,Chrome,Windows,DE,34270024,B5SDZ,39100,615,11,SS
2,Italy,1659.00,1,Firefox,Windows,IT,3069,BQ6RB,31010,430,649,SRL
3,Italy,200.00,1,Safari,Mac OS X,IT,8794,CM5CR,30174,492,614,SAS
4,Italy,1000.00,1,Chrome,Windows,IT,8327,CEFGV,31039,430,505,SRL
5,Italy,120.78,1,Chrome,Windows,IT,8749,CKZYT,31038,430,314,SRL
6,Italy,299.00,0,Other,Linux,IT,8904,082X3,30024,614,342,DI
7,Italy,151.35,0,Samsung Internet,Android,IT,2008,A7TWW,36027,430,316,SRL
8,Italy,1678.00,1,Chrome,Windows,IT,3440,CHRGC,36016,430,830,SRL


In [17]:
test.describe()

,amount,userAgent_isPc,IBAN_bankCode,CAP,SAE,RAE
count,9.000000,9.000000,9.000000e+00,9.000000,9.000000,9.000000
mean,42382.293333,0.777778,3.816368e+06,33717.666667,477.888889,415.777778
std,91142.922544,0.440959,1.142012e+07,3803.032573,80.066292,256.841283
min,120.780000,0.000000,2.008000e+03,30024.000000,430.000000,11.000000
25%,200.000000,1.000000,3.440000e+03,31010.000000,430.000000,314.000000
50%,1000.000000,1.000000,8.749000e+03,31039.000000,430.000000,342.000000
75%,1678.000000,1.000000,8.904000e+03,36027.000000,492.000000,614.000000
max,264986.740000,1.000000,3.427002e+07,39100.000000,615.000000,830.000000


In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  9 non-null      object 
 1   amount                   9 non-null      float64
 2   userAgent_isPc           9 non-null      int64  
 3   userAgent_browserFamily  9 non-null      object 
 4   userAgent_osFamily       9 non-null      object 
 5   IBAN_countryCode         9 non-null      object 
 6   IBAN_bankCode            9 non-null      int64  
 7   SIA                      9 non-null      object 
 8   CAP                      9 non-null      int64  
 9   SAE                      9 non-null      int64  
 10  RAE                      9 non-null      int64  
 11  companyType              9 non-null      object 
dtypes: float64(1), int64(5), object(6)
memory usage: 992.0+ bytes


In [19]:
columns_to_label = ['country','userAgent_browserFamily','userAgent_osFamily','IBAN_countryCode','IBAN_bankCode','CAP','SIA','SAE','RAE','companyType']
for clmn in columns_to_label:
    test[clmn]=test[clmn].apply(str)
    test[clmn] = encoder_dict[clmn].transform(test[clmn])
test.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType
0,0,264986.74,1,0,5,0,364,1482,302,8,4,10
1,6,111345.77,1,0,5,6,376,965,327,25,1,11
2,6,1659.00,1,4,5,17,285,1346,62,8,121,10
3,6,200.00,1,11,3,17,1217,2045,58,19,106,5
4,6,1000.00,1,0,5,17,1088,1781,87,8,99,10


In [20]:
model.predict(test)



array([1, 1, 0, 0, 0, 1, 1, 0, 0], dtype=int64)

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score

for k in range(2,20):
    model=CategoricalNB()
    model.fit(X_train, y_train)
    #print(k)
    #y_score=neigh.predict_proba(X_test)[:,1]
    y_pred=model.predict(X_test)
    print(f1_score(y_test,y_pred)) #diminuisce
    #print(brier_score_loss(y_test,y_score)) #lower better aumenta
    #print(average_precision_score(y_test,y_score,average='micro')) #aumenta
    #print(roc_auc_score(y_test,y_score)) #aumenta
    #print(neigh.score(X_test,y_test)) #diminuisce

0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
0.9937450056487834
